# Spam Classification 🧩📧

This notebook demonstrates a simple but powerful machine learning task: **classifying spam and non spam** using the spamassassin dataset.

The **spamassassin** dataset is a classic benchmark in the field of machine learning.

In this notebook, we will:
- Load and explore the MNIST dataset
- Preprocess the data for model input

In [2]:
import os
import tarfile
import urllib.request

In [3]:
DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

In [4]:
def fetch_spam_dataset(ham_url=HAM_URL, spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
        for filename, url in (("ham.tar.bz2", ham_url),("spam.tar.bz2", spam_url)):
            filepath = os.path.join(spam_path, filename)
            if not os.path.isfile(filepath):
                urllib.request.urlretrieve(url, filepath)
            tar_bz2_file = tarfile.open(filepath)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()

In [5]:
fetch_spam_dataset()

In [6]:
## load dataset
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")

In [ ]:
ham_filename = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filename = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [ ]:
len(ham_filename), len(spam_filename)

(2500, 500)